In [2]:
# File: 02_Model_Training.ipynb

# =============================================================================
# PHASE 3: MODEL BUILDING AND TRAINING
# =============================================================================

# --- 1. SETUP ---

# Import essential libraries
import pandas as pd
from sklearn.model_selection import train_test_split

# Load our preprocessed data from the file we saved in the previous notebook
df_encoded = pd.read_parquet('processed_data/churn_data_encoded.parquet')


# --- 2. SPLITTING THE DATA ---

# Separate features (X) and target (y)
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# --- 3. VERIFICATION ---
print("Setup Complete!")
print("Preprocessed data successfully loaded.")
print("-------------------------------------------------")
print("Shape of training features (X_train):", X_train.shape)
print("Shape of testing features (X_test):", X_test.shape)

Setup Complete!
Preprocessed data successfully loaded.
-------------------------------------------------
Shape of training features (X_train): (5634, 30)
Shape of testing features (X_test): (1409, 30)
